In [1]:
from classes import *

In [2]:
def test(a, b, c):
    qbm = QBM(a, b, c)
    return qbm

In [3]:
a = test(50, 410, True)
a.randomize_coef()
a.read_images()

In [4]:
a.datalen = 0
a.datan = 0
a.sim_run(1)
a.fetch_data()

In [5]:
a.data.shape

(1, 460)

In [6]:
a.add_data()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 460 and the array at index 1 has size 1

In [ ]:
a.data.shape

In [11]:
l = np.arange(15).reshape((3, 5))
m = np.arange(18).reshape((3, 6))
np.concatenate((l, m))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 5 and the array at index 1 has size 6